# Generating Counterfactual Explanations without access to training data
If only the trained model is available but not the training data, DiCE can still be used to generate counterfactual explanations. Below we show an example where DiCE uses only basic metadata about each feature used in the ML model. 

In [ ]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers  # helper functions

# supress deprecation warnings from TF
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
%load_ext autoreload
%autoreload 2

### Defining meta data

We simulate "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult) by providing only meta information about the data: range is provided for continuous features and categories are provided for categorical features. Please note for Python<=3.6, "features" parameter should be provided as an OrderedDict in the same order that was used to train the ML model.

In [ ]:
d = dice_ml.Data(features={
    'age': [17, 90],
    'workclass': ['Government', 'Other/Unknown', 'Private', 'Self-Employed'],
    'education': ['Assoc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Prof-school', 'School', 'Some-college'],
    'marital_status': ['Divorced', 'Married', 'Separated', 'Single', 'Widowed'],
    'occupation': ['Blue-Collar', 'Other/Unknown', 'Professional', 'Sales', 'Service', 'White-Collar'],
    'race': ['Other', 'White'],
    'gender': ['Female', 'Male'],
    'hours_per_week': [1, 99]},
                 outcome_name='income')

### Loading trained ML model

Below, we use a trained ML model which produces high accuracy on test datasets, comparable to other popular baselines. This sample trained model comes in-built with our package.

The variable *backend* below indicates the implementation type of DiCE we want to use. We use TensorFlow 1.x in the notebooks with backend='TF1'. You can set backend to 'TF2' or 'PYT' to use DiCE with TensorFlow 2.x or with PyTorch respectively. We want to note that the time required to find counterfactuals with Tensorflow 2.x's eager style of execution is significantly greater than that with TensorFlow 1.x's graph execution.

In [ ]:
backend = 'TF'+tf.__version__[0]  # TF1
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
m = dice_ml.Model(model_path=ML_modelpath, backend=backend)

### Generate diverse counterfactuals

In [ ]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [ ]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age': 22,
                  'workclass': 'Private',
                  'education': 'HS-grad',
                  'marital_status': 'Single',
                  'occupation': 'Service',
                  'race': 'White',
                  'gender': 'Female',
                  'hours_per_week': 45}

In [ ]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

In [ ]:
# visualize the results
dice_exp.visualize_as_dataframe(show_only_changes=True)

When the training data is available, by default, the distance between two values of a continuous feature is scaled by the inverse median absolute deviation (MAD) of the feature using the training data. This is done to capture the relative prevalence of observing a continuous feature at a particular value, as discussed in [our paper](https://arxiv.org/pdf/1905.07697.pdf). However, when there is no access to the training data, as in the above case, no scaling is done and hence all features are weighted equally in the normalized form. As a result, the counterfactuals generated above are different from those in [DiCE_getting_started](DiCE_getting_started.ipynb) notebook where the training data was available. Nonetheless, you can manually provide the scaling constants through a parameter *feature_weights* to the *generate_counterfactuals()* method as shown in this [advanced notebook](DiCE_with_advanced_options.ipynb), or you can provide the MADs directly to the data interface *dice_ml.Data* if you know them.